In [1]:
import numpy as np
import spatialdata as sd

from multiplex_pipeline.core_segmentation.controller import SegmentationController
from multiplex_pipeline.core_segmentation.segmenters import CellposeSegmenter

c:\Users\KMK280\AppData\Local\miniconda3\envs\cellpose-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


## Cellpose nuclear segmentation

In [5]:
sdata_path = r'R:\Wayne\BLCA\BLCA-1B_Analysis\temp\Core_015.zarr' 

In [6]:
cellpose_segmenter = CellposeSegmenter(model_type = 'cyto2', diameter = 40)
segmentation_controller = SegmentationController(cellpose_segmenter, resolution_level = 0)

In [7]:
segmentation_controller.segment_spatial_data(sdata_path, channels = ['DAPI'], mask_name = "cellpose_nucleus")

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\cellpose-env\Lib\site-packages\zarr\creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04


In [8]:
# check content of the SpatialData object
sdata = sd.read_zarr(sdata_path)
sdata

version mismatch: detected: RasterFormatV02, requested: FormatV04
version mismatch: detected: RasterFormatV02, requested: FormatV04


SpatialData object, with associated Zarr store: \\share.files.pitt.edu\HSIT-Stallaert-Lab\Wayne\BLCA\BLCA-1B_Analysis\temp\Core_015.zarr
├── Images
│     ├── 'CD3': DataTree[cyx] (1, 5824, 5824), (1, 2912, 2912), (1, 1456, 1456)
│     └── 'DAPI': DataTree[cyx] (1, 5824, 5824), (1, 2912, 2912), (1, 1456, 1456)
└── Labels
      └── 'cellpose_nucleus': DataTree[yx] (5824, 5824), (2912, 2912), (1456, 1456)
with coordinate systems:
    ▸ 'global', with elements:
        CD3 (Images), DAPI (Images), cellpose_nucleus (Labels)

## Instanseg for nucleus and cell segmentation using multiple channels